In [2]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys
import pandas as pd
import json

In [19]:
venue_df=pd.DataFrame(columns=['Name','LikesCount','RetweetCount'])
print venue_df

Empty DataFrame
Columns: [Name, LikesCount, RetweetCount]
Index: []


In [60]:
def load_api():
    consumer_key = 'rqXF1X0aqHngHqi7ohWVBcd4R'
    consumer_secret = 'JLy4o5vmgy8ignlFl122Y1qH53t3aCD7Veb5H208rdRxf9c3Gm'
    access_token = '3094362888-AuKPdG8xwKyCFKeQlDaryNxywVNcXOJueaijeoU'
    access_secret = 'i9lziRbZwoDnPXphtmrob6X9Beb8bAoKitdXEjwNZcQbS'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    # load the twitter API via tweepy
    return tweepy.API(auth)

    
def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
    ''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''

    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
                                max_id=str(max_id-1),
                                geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id


def get_tweet_id(api, date='', days_ago=9, query='a'):
    ''' Function that gets the ID of a tweet. This ID can then be
        used as a 'starting point' from which to search. The query is
        required and has been set to a commonly used word by default.
        The variable 'days_ago' has been initialized to the maximum
        amount we are able to search back in time (9).'''

    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id


def write_tweets(tweets, filename):
    ''' Function that appends tweets to a file. '''

    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')
            
    
    
def populate_tweet_df(tweets):
    df = pd.DataFrame(columns=['text','screenname','location','country_code','likes','retweets','retweet_likes','long','latt'])
    
    retweet_likes_lst=[]
    
    df['text'] = list(map(lambda tweet: tweet['text'], tweets))
    
    df['screenname'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets))
 
    for tweet in tweets:
        if tweet.get("retweeted_status"):
            retweet_likes_lst.append(tweet['retweeted_status']['favorite_count'])
        else:
            retweet_likes_lst.append(0)
    df['retweet_likes']=retweet_likes_lst

    df['location'] = list(map(lambda tweet: tweet['user']['location'], tweets))
 
    df['country_code'] = list(map(lambda tweet: tweet['place']['country_code']
                                  if tweet['place'] != None else '', tweets))
    
    df['likes'] = list(map(lambda tweet: tweet['favorite_count'], tweets))
     
    df['retweets'] = list(map(lambda tweet: tweet['retweet_count'], tweets))    
 
    df['long'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][0]
                        if tweet['coordinates'] != None else 'NaN', tweets))
 
    df['latt'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][1]
                        if tweet['coordinates'] != None else 'NaN', tweets))
 
    return df

def aggregate(data,search_phrase):
    print "Name:",search_phrase
    venue_name_lst.append(search_phrase)
    
    tweet_likes=data['likes'].sum()
    retweet_likes=data['retweet_likes'].sum()
    tot_likes=tweet_likes+retweet_likes
    likes_lst.append(tot_likes)
    
    retweets_lst.append(data['retweets'].sum())

In [61]:
likes_lst=[]
retweets_lst=[]
venue_name_lst=[]

def main():
    ''' This is a script that continuously searches for tweets
        that were created over a given number of days. The search
        dates and search phrase can be changed below. '''

    ''' search variables: '''
    search_phrases = ['Isle Royale National Park', 'Joshua Tree National Park', 'Katmai National Park and Preserve', 'Kenai Fjords National Park', 'Kings Canyon National Park', 'Kobuk Valley National Park', 'Lake Clark National Park', 'Lassen Volcanic National Park', 'Mammoth Cave National Park', 'Mesa Verde National Park', 'Mount Rainier National Park', 'National Park of American Samoa', 'North Cascades National Park', 'Olympic National Park', 'Petrified Forest National Park', 'Pinnacles National Park', 'Redwood National Park', 'Rocky Mountain National Park', 'Saguar National Park', 'Sequoia National Park', 'Shenandoah National Park', 'Theodore Roosevelt National Park', 'Virgin Islands National Park', 'Voyageurs National Park', 'Wind Cave National Park', 'Wrangell-St. Elias National Park ad Preserve', 'Yellowstone National Park', 'Yosemite National Park', 'Zion National Park']
    #search_phrases = ['Yellowstone'];
    time_limit = 1.5                           # runtime limit in hours
    max_tweets = 200                           # number of tweets per search (will be
                                               # iterated over) - maximum is 100
    #min_days_old, max_days_old = 6, 7          # search limits e.g., from 7 to 8
    min_days_old, max_days_old = 1,8
                                               # gives current weekday from last week,
                                               # min_days_old=0 will search from right now
    USA = '39.8,-95.583068847656,2500km'       # this geocode includes nearly all American
                                               # states (and a large portion of Canada)
    

    # loop over search items,
    # creating a new file for each
    for search_phrase in search_phrases:

        print "Search phrase =",search_phrase

        ''' other variables '''
        #name = search_phrase.split()[0]
        name = search_phrase
        json_file_root = name + '/' + name
        
        if (os.path.exists(name) == False):
            os.makedirs(os.path.dirname(json_file_root), 0755)
        read_IDs = False
        
        # open a file in which to store the tweets
        if max_days_old - min_days_old == 1:
            d = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        else:
            d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
            d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
                  d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
        json_file = json_file_root + '_' + day + '.json'
        if os.path.isfile(json_file):
            #print('Appending tweets to file named: ',json_file)
            read_IDs = True
        
        # authorize and load the twitter API
        api = load_api()
        
        # set the 'starting point' ID for tweet collection
        if read_IDs:
            # open the json file and get the latest tweet ID
            with open(json_file, 'r') as f:
                lines = f.readlines()
                max_id = json.loads(lines[-1])['id']
                #print('Searching from the bottom ID in file')
        else:
            # get the ID of a tweet that is min_days_old
            if min_days_old == 0:
                max_id = -1
            else:
                max_id = get_tweet_id(api, days_ago=(min_days_old-1))
        # set the smallest ID to search for
        since_id = get_tweet_id(api, days_ago=(max_days_old-1))
        #print('max id (starting point) =', max_id)
        #print('since id (ending point) =', since_id)
        
        ''' tweet gathering loop  '''
        start = dt.datetime.now()
        end = start + dt.timedelta(hours=time_limit)
        count, exitcount = 0, 0
        #like_count = 0
        while dt.datetime.now() < end:
            count += 1
            #print('count =',count)
            # collect tweets and update max_id
            tweets, max_id = tweet_search(api, search_phrase, max_tweets,
                                          max_id=max_id, since_id=since_id,
                                          geocode=USA)

            
            # write tweets to file in JSON format
            if tweets:
                write_tweets(tweets, json_file)
                exitcount = 0
            else:
                exitcount += 1
                if exitcount == 3:
                    if search_phrase == search_phrases[-1]:
                        break
                    else:
                        #print('Maximum number of empty tweet strings reached - breaking')
                        break
        
        file = json_file
        tweets = []
        if os.path.isfile(json_file):
            with open(file, 'r') as f:
                    for line in f.readlines():
                        tweets.append(json.loads(line))

            tweetsDf  = populate_tweet_df(tweets)
            aggregate(tweetsDf,search_phrase)
        
#dataFrame.to_csv("twitter.csv", sep='\t', encoding='utf-8')
        
if __name__ == "__main__":
    main()

    
venue_df['Name']=venue_name_lst
venue_df['LikesCount']=likes_lst
venue_df['RetweetCount']=retweets_lst

print venue_df

Search phrase = Yellowstone
('search limit (start/stop):', datetime.datetime(2017, 10, 1, 23, 59, 59))
('found', 0, 'tweets')
no tweets found
('found', 0, 'tweets')
no tweets found
('found', 0, 'tweets')
no tweets found
Name: Yellowstone
          Name  LikesCount  RetweetCount
0  Yellowstone     1164398        790777


In [173]:
venue_df.to_csv("twitter.csv", sep= '\t' , encoding = "utf-8")

In [22]:
print tweetsDf.head()

NameError: name 'tweetsDf' is not defined

In [17]:
name = 'Yellowstone'
    
if (os.path.exists(name) == False):
    print "Does not exist"
else:
    print "Exists"


Exists
